In [196]:
import torch
from torch import nn
import torch.nn.functional as F

class Encoder(nn.Module):
    def __init__(self, input_channels, latent_dim):
        super(Encoder, self).__init__()
        self.conv1 = nn.Conv2d(input_channels, 32, kernel_size=3, stride=2, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1)
        # Dynamically calculate the flattened size
        self.flatten_size = None
        self.fc1 = nn.Linear(1, 64)  # Placeholder, will be initialized in forward
        self.fc_mean = nn.Linear(64, latent_dim)
        self.fc_log_var = nn.Linear(64, latent_dim)
        self.initialized = False

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        if not self.initialized:
            self.flatten_size = x.shape[1] * x.shape[2] * x.shape[3]
            self.fc1 = nn.Linear(self.flatten_size, 64)  # Proper initialization
            self.initialized = True
        x = torch.flatten(x, start_dim=1)
        x = F.relu(self.fc1(x))
        mean = self.fc_mean(x)
        log_var = self.fc_log_var(x)
        return mean, log_var

class Decoder(nn.Module):
    def __init__(self, latent_dim, output_channels):
        super(Decoder, self).__init__()
        # Assuming the latent dimension reshapes to something that can be scaled up to the input dimensions
        self.fc = nn.Linear(latent_dim, 128 * 17 * 1)  # Adjust the multiplication factors to fit your specific structure
        self.conv2d_t1 = nn.ConvTranspose2d(128, 64, kernel_size=3, stride=2, padding=1, output_padding=1)
        self.conv2d_t2 = nn.ConvTranspose2d(64, 32, kernel_size=3, stride=2, padding=1, output_padding=1)
        self.conv2d_t3 = nn.ConvTranspose2d(32, output_channels, kernel_size=(1,1), stride=2, padding=(51,2), output_padding=(0,1))

    def forward(self, z):
        z = F.relu(self.fc(z))
        z = z.view(-1, 128, 17, 1)  # Adjusted to match the output of the encoder's last conv layer
        z = F.relu(self.conv2d_t1(z))
        z = F.relu(self.conv2d_t2(z))
        reconstruction = torch.sigmoid(self.conv2d_t3(z))
        return reconstruction

# class VAE(nn.Module):
#     def __init__(self, input_channels, latent_dim, output_channels):
#         super(VAE, self).__init__()
#         self.encoder = Encoder(input_channels, latent_dim)
#         self.decoder = Decoder(latent_dim, output_channels)

#     def reparameterize(self, mean, log_var):
#         std = torch.exp(0.5 * log_var)
#         eps = torch.randn_like(std)
#         return mean + eps * std

#     def forward(self, x):
#         mean, log_var = self.encoder(x)
#         z = self.reparameterize(mean, log_var)
#         return self.decoder(z), mean, log_var
    
class SupervisedVAE(nn.Module):
    def __init__(self, input_channels, latent_dim, output_channels, num_classes=11):
        super(SupervisedVAE, self).__init__()
        self.encoder = Encoder(input_channels, latent_dim)
        self.decoder = Decoder(latent_dim, output_channels)
        self.classifier = nn.Linear(latent_dim, num_classes)  # Classification layer

    def forward(self, x):
        mean, log_var = self.encoder(x)
        z = self.reparameterize(mean, log_var)
        reconstruction = self.decoder(z)
        class_logits = self.classifier(z)
        return reconstruction, mean, log_var, class_logits

    def reparameterize(self, mean, log_var):
        std = torch.exp(0.5 * log_var)
        eps = torch.randn_like(std)
        return mean + eps * std

In [197]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import stft

data_path = '../datasets/UPFallCompleteDataSet.csv'
data = pd.read_csv(data_path, skiprows=1, usecols=[0, 15, 16, 17, 44])  # Adjust column indices as needed

# Rename columns for clarity
data.columns = ['timestamp', 'acc_x', 'acc_y', 'acc_z', 'activity']
# Drop rows with any NaN values
data = data.dropna()

data['timestamp'] = pd.to_datetime(data['timestamp'])

data

,timestamp,acc_x,acc_y,acc_z,activity
0,2018-07-04 12:04:17.738369,0.146,0.895,0.367,1
1,2018-07-04 12:04:17.790509,0.146,0.895,0.367,1
2,2018-07-04 12:04:17.836632,0.178,0.896,0.373,1
3,2018-07-04 12:04:17.885262,0.160,0.895,0.372,1
4,2018-07-04 12:04:17.945423,0.160,0.895,0.372,1
...,...,...,...,...,...
294673,2018-07-12 12:18:28.783680,-0.377,-0.095,0.989,11
294674,2018-07-12 12:18:28.832811,-0.376,-0.090,0.998,11
294675,2018-07-12 12:18:28.892470,-0.376,-0.090,0.998,11
294676,2018-07-12 12:18:29.025324,-0.381,-0.101,0.992,11


In [198]:
from scipy.stats import mode

def create_overlapping_windows_with_labels(signals, labels, window_size, overlap_size):
    step_size = window_size - overlap_size
    windowed_signals = []
    windowed_labels = []
    total_windows = 0

    for start in range(0, len(signals[0]) - window_size + 1, step_size):  # Assumes all signals are the same length
        end = start + window_size
        window = [signal[start:end] for signal in signals]
        label_window = labels[start:end]
        if len(window[0]) == window_size and len(label_window) == window_size:
            windowed_signals.append(window)
            # Using mode of labels in the window as the window label
            windowed_label = mode(label_window)[0]
            windowed_labels.append(windowed_label)
            total_windows += 1
        else:
            print(f"Skipped window from {start} to {end} due to incorrect size.")

    return np.array(windowed_signals), np.array(windowed_labels)

def generate_multi_channel_spectrogram(windows, fs, window_type, nperseg, noverlap, nfft):
    spectrograms = []
    for window in windows:
        # Assuming window is a 2D array with shape (window_size, num_channels)
        window_spectrograms = []
        for signal in window:  # Transpose to iterate over channels
            _, _, Zxx = stft(signal, fs=fs, window=window_type, nperseg=nperseg, noverlap=noverlap, nfft=nfft)
            spectrogram = np.log(np.abs(Zxx) + 1e-8)  # Convert to dB scale
            # Normalize the spectrogram
            mean = np.mean(spectrogram)
            std = np.std(spectrogram)
            spectrogram = (spectrogram - mean) / std
            window_spectrograms.append(spectrogram)
        # Stack to form a multi-channel spectrogram for this window
        spectrograms.append(np.stack(window_spectrograms, axis=0))
    return np.array(spectrograms)

def plot_simple_spectrogram(spectrogram, channel=0, title="Spectrogram"):
    # spectrogram should be a 3D array: [frequency, time, channel]
    plt.figure(figsize=(10, 4))
    # Display the spectrogram of a specific channel
    plt.imshow(spectrogram[:, :, channel].T, aspect='auto', origin='lower')
    plt.colorbar(label='Intensity (dB)')
    plt.title(title)
    plt.ylabel('Sample Index')
    plt.xlabel('Time Index')
    plt.show()

In [199]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim

# Example parameters
window_size = 45 # aroung 4sec of data
overlap_size = 22
fs = 15  # Sampling frequency is aroung 12.85

signals = [data['acc_x'].values, data['acc_y'].values, data['acc_z'].values]
activities = data['activity'].values - 1 # make it zero-indexed

# Create overlapping windows for each signal and stack them to form multi-dimensional windows
windowed_signals, windowed_labels = create_overlapping_windows_with_labels(signals, activities, window_size, overlap_size)

# Generate spectrograms for each window
window_type='hann'
nperseg=32
noverlap=16
nfft=64
spectrograms = generate_multi_channel_spectrogram(windowed_signals, fs, window_type, nperseg, noverlap, nfft)

# Split data
X_train, X_test, y_train, y_test = train_test_split(spectrograms, windowed_labels, test_size=0.3, random_state=42)


In [200]:
class EarlyStopping:
    def __init__(self, patience=5, verbose=False, delta=0):
        """
        Args:
            patience (int): How many epochs to wait after last time validation loss improved.
                            Default: 5
            verbose (bool): If True, prints a message for each validation loss improvement. 
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
        """
        self.patience = patience
        self.verbose = verbose
        self.delta = delta
        self.best_score = None
        self.epochs_no_improve = 0
        self.early_stop = False

    def __call__(self, val_loss):
        score = -val_loss

        if self.best_score is None:
            self.best_score = score
        elif score < self.best_score + self.delta:
            self.epochs_no_improve += 1
            if self.epochs_no_improve >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.epochs_no_improve = 0

        if self.verbose and self.epochs_no_improve == 0:
            print(f'Validation loss decreased ({self.best_score - score:.6f} --> {score:.6f}).  Saving model ...')


In [201]:
class Data(torch.utils.data.Dataset):
  def __init__(self, X, y, device='cpu'):
    self.x_data = torch.tensor(X, dtype=torch.float32).to(device)
    self.y_data = torch.tensor(y, dtype=torch.float32).to(device)

  def __len__(self):
    return len(self.x_data)

  def __getitem__(self, idx):
    seq_in = self.x_data[idx]
    target = self.y_data[idx]
    sample = { 'spectrograms' : seq_in, 'label' : target }
    return sample

# Convert to PyTorch tensors and create DataLoaders
train_data = Data(X_train, y_train)
test_data = Data(X_test, y_test)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

In [202]:
from torch.optim.lr_scheduler import StepLR

# Loss function
def supervised_vae_loss(recon_x, x, mean, log_var, logits, labels):
    # print(f"Recon_x size: {recon_x.size()}, x size: {x.size()}")
    MSE = F.mse_loss(recon_x, x, reduction='sum')
    KLD = -0.5 * torch.sum(1 + log_var - mean.pow(2) - log_var.exp())
    labels = labels.long()
    CE = F.cross_entropy(logits, labels)
    return MSE, KLD, CE, MSE/10000 + KLD + CE

# Train Function
def train_supervised_vae(model, data_loader, epochs, learning_rate=1e-3, device='cpu'):
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    scheduler = StepLR(optimizer, step_size=10, gamma=0.1)
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        total_MSE = 0
        total_KLD = 0
        total_CE = 0
        for batch in data_loader:
            data = batch['spectrograms'].to(device)
            labels = batch['label'].to(device)
            optimizer.zero_grad()
            recon_batch, mean, log_var, class_logits = model(data)
            MSE, KLD, CE, loss = supervised_vae_loss(recon_batch, data, mean, log_var, class_logits, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss
            total_MSE += MSE
            total_CE += CE
            total_KLD += KLD
        
        # scheduler.step()
        average_loss = total_loss / len(data_loader)
        average_MSE = total_MSE / len(data_loader)
        average_KLD = total_KLD / len(data_loader)
        average_CE = total_CE / len(data_loader)
        print(f'Epoch {epoch+1}, Average Loss: {average_loss:.4f}, Average KLD: {average_KLD:.8f}, Weighted Average MSE: {average_MSE:.4f} / 10000 = {average_MSE/10000:.4f}, Average CE: {average_CE:.4f}')
        #  / 1.5 = {average_KLD/1.5:.4f}

In [203]:
# from torch.optim.lr_scheduler import StepLR

# # Loss function
# def supervised_vae_loss(recon_x, x, mean, log_var, logits, labels):
#     print(f"Recon_x size: {recon_x.size()}, x size: {x.size()}")
#     print(recon_x, x)
#     MSE = F.mse_loss(recon_x, x, reduction='sum')
#     KLD = -0.5 * torch.sum(1 + log_var - mean.pow(2) - log_var.exp())
#     labels = labels.long()
#     CE = F.cross_entropy(logits, labels)
#     return MSE + KLD + CE

# # Train Function
# def train_supervised_vae(model, data_loader, val_loader, epochs, learning_rate=1e-3, device='cuda'):
#     optimizer = optim.Adam(model.parameters(), lr=learning_rate)
#     scheduler = StepLR(optimizer, step_size=10, gamma=0.1)
#     early_stopping = EarlyStopping(patience=5, verbose=True)
#     min_loss = 100000000
#     current_loss = 0
    
#     for epoch in range(epochs):
#         model.train()
#         total_loss = 0
#         for batch in data_loader:
#             data = batch['spectrograms'].to(device)
#             labels = batch['label'].to(device)
#             optimizer.zero_grad()
#             recon_batch, mean, log_var, class_logits = model(data)
#             current_loss = supervised_vae_loss(recon_batch, data, mean, log_var, class_logits, labels)
#             current_loss.backward()
#             optimizer.step()
#             total_loss += current_loss.item()
#             if current_loss < min_loss:
#                 min_loss = current_loss
        
#         # Validation phase
#         model.eval()
#         total_val_loss = 0
#         current_val_loss = 0
#         with torch.no_grad():
#             for batch in val_loader:
#                 data = batch['spectrograms'].to(device)
#                 labels = batch['label'].to(device)
#                 recon_batch, mean, log_var, class_logits = model(data)
#                 current_val_loss = supervised_vae_loss(recon_batch, data, mean, log_var, class_logits, labels)
#                 total_val_loss += current_val_loss.item()

#         # Scheduler and Early Stopping
#         val_loss_avg = total_val_loss / len(val_loader)
#         early_stopping(val_loss_avg)
#         # scheduler.step()
#         average_loss = total_loss / len(data_loader)
#         print(f'Epoch {epoch+1}, Average Loss: {average_loss:.4f}, Min Loss: {min_loss:.4f}')
        
#         if early_stopping.early_stop:
#             print(f"Early stopping. Current Loss: {current_val_loss:.4f}")
#             break

In [204]:
supervised_vae_model = SupervisedVAE(input_channels=3, latent_dim=20, output_channels=3)
device = torch.device("cpu" if torch.cuda.is_available() else "cpu")
supervised_vae_model.to(device)
train_supervised_vae(supervised_vae_model, train_loader, epochs=100, learning_rate=0.001)

Epoch 1, Average Loss: 3.8540, Average KLD: 0.07786067, Weighted Average MSE: 13695.3232 / 10000 = 1.3695, Average CE: 2.4066
Epoch 2, Average Loss: 3.5642, Average KLD: 0.00131312, Weighted Average MSE: 13057.1846 / 10000 = 1.3057, Average CE: 2.2572
Epoch 3, Average Loss: 3.4509, Average KLD: 0.00075580, Weighted Average MSE: 12700.1523 / 10000 = 1.2700, Average CE: 2.1801
Epoch 4, Average Loss: 3.3847, Average KLD: 0.00049653, Weighted Average MSE: 12441.8262 / 10000 = 1.2442, Average CE: 2.1400
Epoch 5, Average Loss: 3.3463, Average KLD: 0.00037672, Weighted Average MSE: 12254.4453 / 10000 = 1.2254, Average CE: 2.1205
Epoch 6, Average Loss: 3.3262, Average KLD: 0.00030121, Weighted Average MSE: 12116.3145 / 10000 = 1.2116, Average CE: 2.1143
Epoch 7, Average Loss: 3.3093, Average KLD: 0.00023580, Weighted Average MSE: 12013.2715 / 10000 = 1.2013, Average CE: 2.1078
Epoch 8, Average Loss: 3.3014, Average KLD: 0.00023040, Weighted Average MSE: 11935.5146 / 10000 = 1.1936, Average CE:

In [207]:
def predict(model, data_loader, device):
    model.eval()  # Set the model to evaluation mode
    predictions = []
    reconstructions = []
    with torch.no_grad():  # Turn off gradients for validation, saves memory and computations
        for batch in data_loader:
            inputs = batch['spectrograms'].to(device)
            labels = batch['label'].to(device)
            recon, _, _, logits = model(inputs)
            preds = torch.argmax(logits, dim=1)  # Get the class predictions
            predictions.extend(preds.cpu().numpy())  # Store predictions
            reconstructions.append(recon.cpu())  # Store reconstructions
    return predictions, reconstructions

from sklearn.metrics import accuracy_score, classification_report

# Assuming `test_loader` is your DataLoader for the test set
predictions, reconstructions = predict(supervised_vae_model, test_loader, device)

# Assuming `true_labels` are your actual labels for the test dataset
print("Accuracy:", accuracy_score(y_test, predictions))
print(classification_report(y_test, predictions))

# For reconstruction error, if applicable:
recon_errors = [F.mse_loss(r, inputs).item() for r, inputs in zip(reconstructions, test_loader.dataset.x_data)]
print("Average Reconstruction Error:", np.mean(recon_errors))

# import matplotlib.pyplot as plt

# def plot_reconstructions(inputs, recons, n=10):
#     plt.figure(figsize=(10, 4))
#     for i in range(n):
#         plt.subplot(2, n, i + 1)
#         plt.imshow(inputs[i].squeeze().numpy())  # Adjust based on your data
#         plt.title("Original")
#         plt.axis('off')

#         plt.subplot(2, n, n + i + 1)
#         plt.imshow(recons[i].squeeze().numpy())  # Adjust based on your data
#         plt.title("Reconstructed")
#         plt.axis('off')
#     plt.show()

# # Select first n examples to plot
# plot_reconstructions(test_loader.dataset.x_data[:10], reconstructions[:10])



Accuracy: 0.1896462018730489
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       129
           1       0.00      0.00      0.00       112
           2       0.00      0.00      0.00       107
           3       0.00      0.00      0.00       123
           4       0.00      0.00      0.00       123
           5       0.20      0.33      0.25       723
           6       0.18      0.42      0.26       681
           7       0.19      0.27      0.22       693
           8       0.00      0.00      0.00       109
           9       0.00      0.00      0.00       363
          10       0.15      0.02      0.04       681

    accuracy                           0.19      3844
   macro avg       0.07      0.10      0.07      3844
weighted avg       0.13      0.19      0.14      3844

Average Reconstruction Error: 0.9195633545394771


c:\Users\Lione\miniconda3\envs\IMED2024\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Lione\miniconda3\envs\IMED2024\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Lione\miniconda3\envs\IMED2024\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(resu